In [1]:
from multiprocessing import Pool, Process, Queue
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re

In [2]:
# генерация random user agents для обхода блокировок от сайта
!pip install random_user_agent
import random
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   
user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=300)
user_agents = user_agent_rotator.get_user_agents()
user_agent = user_agent_rotator.get_random_user_agent()

agents_list = []
for i in range(150):
    i = user_agent_rotator.get_random_user_agent()
    agents_list.append(i)

     |████████████████████████████████| 8.2MB 7.4MB/s 


In [7]:
# сбор ссылок по страницам
%%time
range_n = [i for i in range(952)]


def fill_urls(number):
    links = []
    http = 'http://sakha.ysia.ru/page/{}/?filter_by=popular'.format(number)
    page = requests.get(http, headers = {'User-Agent': random.choice(agents_list)})
    soup = bs(page.text, 'html.parser')
    all_posts = soup.find('div', class_= 'td-ss-main-content')
    posts = all_posts.find_all('div', class_ = 'item-details')
    for x in posts:
        link = x.find('a')['href']
        links.append(link)
    return links


big_links = []
p = Pool(24)


for links in p.map(fill_urls, range_n):
    big_links.extend(links)
print('number of news:', len(big_links))

p.terminate()
p.join()

number of news: 9511
CPU times: user 411 ms, sys: 259 ms, total: 670 ms
Wall time: 55.8 s


In [9]:
# запуск сбора данных из ссылок
%%time


def parse(link):
    dataframe = []
    page = requests.get(link, headers = {'User-Agent': random.choice(agents_list)})
    if page.status_code==200:
        soup = bs(page.text, 'html.parser')
        if soup is not None:
            content = soup.find('div', class_='td-post-content')
            if soup.find('time') is not None:
                date = soup.find('time').text.split(' ')[0]
                if soup.find('h1', class_= 'entry-title') is not None:
                    title = soup.find('h1', class_= 'entry-title').text
                    text = ''
                    if content.find_all('p') is not None:
                        all_text = content.find_all('p')
                        for i in all_text:
                            text += ' '+ i.text
                        if text is not None:
                            text = re.sub(r"\s{2,}", "", text).replace('\n', '')
                            dataframe.append((date, link, title, text))
                            return dataframe


df_info = []
p = Pool(24)

for dataframe in p.map(parse, big_links):
    if dataframe:
        df_info.extend(dataframe)
    else: 
        continue
p.terminate()
p.join()

len(df_info)

CPU times: user 3.64 s, sys: 1.42 s, total: 5.06 s
Wall time: 9min 2s


In [10]:
df = pd.DataFrame(df_info, columns=['date', 'link', 'title', 'all_text'])
df.to_csv('sakha_ysia.csv', index=False)